In [2]:
from smt.surrogate_models import KRG
from smt_explainability.problems import MixedCantileverBeam
from smt.design_space import (
    DesignSpace,
    FloatVariable,
    CategoricalVariable,
)
from smt.surrogate_models import (
    KPLS,
    MixIntKernelType,
    MixHrcKernelType,
)
from smt.applications.mixed_integer import MixedIntegerKrigingModel

from smt_explainability.pdp.partial_dependence_display import PartialDependenceDisplay
from smt_explainability.pdp.pd_feature_importance_display import PDFeatureImportanceDisplay
from smt_explainability.pdp.pd_interaction_display import PDFeatureInteractionDisplay

from sklearn.metrics import mean_squared_error
import numpy as np
import time

In [3]:
ndoe = 200
n_train = int(0.8 * ndoe)
fun = MixedCantileverBeam()
# Name of the features
feature_names = [r'$\tilde{I}$', r'$L$', r'$S$']
# Index for categorical features
categorical_feature_indices = [0]
# Design space
ds = DesignSpace([
    CategoricalVariable(values=[str(i + 1) for i in range(12)]),
    FloatVariable(10.0, 20.0),
    FloatVariable(1.0, 2.0),
])
# create mapping for the categories
categories_map = dict()
inverse_categories_map = dict()
for feature_idx in categorical_feature_indices:
    categories_map[feature_idx] = {
        i: value for i, value in enumerate(ds._design_variables[feature_idx].values)
    }
    inverse_categories_map[feature_idx] = {
        value: i for i, value in enumerate(ds._design_variables[feature_idx].values)
    }

X = fun.sample(ndoe)
y = fun(X)

X_tr, y_tr = X[:n_train, :], y[:n_train]
X_te, y_te = X[n_train:, :], y[n_train:]

class GroundTruthModel:
    def predict_values(self, X):
        return fun(X)
    
gtm = GroundTruthModel()

In [4]:
sm = MixedIntegerKrigingModel(
    surrogate=KPLS(
        design_space=ds,
        categorical_kernel=MixIntKernelType.HOMO_HSPHERE,
        hierarchical_kernel=MixHrcKernelType.ARC_KERNEL,
        theta0=np.array([4.43799547e-04, 4.39993134e-01, 1.59631650e+00]),
        corr="squar_exp",
        n_start=1,
        cat_kernel_comps=[2],
        n_comp=2,
        print_global=False,
        ),
    )

start_time = time.time()
sm.set_training_values(X_tr, np.array(y_tr))
sm.train()
print("run time (s):", time.time() - start_time)

print("Surrogate model")
y_pred = sm.predict_values(X_te)
rmse = mean_squared_error(y_te, y_pred, squared=False)
rrmse = rmse / y_te.mean()
print(f"RMSE: {rmse:.4f}")
print(f"rRMSE: {rrmse:.4f}")

/Users/muhammaddaffarobani/miniconda3/envs/smt_ex_310/lib/python3.10/site-packages/smt/applications/mixed_integer.py:208: UserWarning: TNC not available yet for mixed integer handling. Switching to Cobyla
  warnings.warn(


run time (s): 29.985137224197388
Surrogate model
RMSE: 0.0001
rRMSE: 0.1204


/Users/muhammaddaffarobani/miniconda3/envs/smt_ex_310/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [5]:
model = gtm

gt_pd_importance = PDFeatureImportanceDisplay.from_surrogate_model(
    model, X_tr, feature_names=feature_names, categorical_feature_indices=categorical_feature_indices
)
gt_pd_importance_plot = gt_pd_importance.plot(figsize=[8, 4])
gt_pd_importance_plot.savefig('gt_pd_importance_mixed.png')

In [6]:
model = sm

sm_pd_importance = PDFeatureImportanceDisplay.from_surrogate_model(
    model, X_tr, feature_names=feature_names, categorical_feature_indices=categorical_feature_indices
)
sm_pd_importance_plot = sm_pd_importance.plot(figsize=[8, 4])
sm_pd_importance_plot.savefig('sm_pd_importance_mixed.png')

In [7]:
model = gtm

features = [i for i in range(X_tr.shape[1])]

gt_pdd = PartialDependenceDisplay.from_surrogate_model(
    model, 
    X_tr, 
    features, 
    categorical_feature_indices=categorical_feature_indices, 
    feature_names=feature_names,
    grid_resolution=20,
    kind='both',
    categories_map=categories_map,
    )

gt_pdd_plot_1d = gt_pdd.plot(centered=True)
gt_pdd_plot_1d.savefig('gt_pdd_1d_mixed.png')

In [8]:
%%time
model = sm

features = [i for i in range(X_tr.shape[1])]

pdd = PartialDependenceDisplay.from_surrogate_model(
    model, 
    X_tr, 
    features, 
    categorical_feature_indices=categorical_feature_indices, 
    feature_names=feature_names,
    grid_resolution=20,
    kind='both',
    categories_map=categories_map,
    )

pdd_plot_1d = pdd.plot(centered=True)
pdd_plot_1d.savefig('sm_pdd_1d_mixed.png')

CPU times: user 29.5 s, sys: 1min 13s, total: 1min 43s
Wall time: 13.8 s


In [9]:
model = gtm
features = [(0, 1), (1, 2)]

gt_pdd_2d = PartialDependenceDisplay.from_surrogate_model(
    model, 
    X_tr, 
    features, 
    categorical_feature_indices=categorical_feature_indices, 
    feature_names=feature_names,
    grid_resolution=10,
    kind='both',
    categories_map=categories_map,
    )

gt_pdd_plot_2d = gt_pdd_2d.plot(centered=True, figsize=[20, 5])
gt_pdd_plot_2d.savefig('gt_pdd_2d_mixed.png')

In [10]:
%%time
model = sm
features = [(0, 1), (1, 2)]

pdd_2d = PartialDependenceDisplay.from_surrogate_model(
    model, 
    X_tr, 
    features, 
    categorical_feature_indices=categorical_feature_indices, 
    feature_names=feature_names,
    grid_resolution=10,
    kind='both',
    categories_map=categories_map,
    )

pdd_plot_2d = pdd_2d.plot(centered=True, figsize=[20, 5])
pdd_plot_2d.savefig('sm_pdd_2d_mixed.png')

CPU times: user 1min 58s, sys: 5min 10s, total: 7min 9s
Wall time: 56.9 s
